<a href="https://colab.research.google.com/github/robertsarnet/IDS-project/blob/main/IDS_project_user_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Connecting to Google Drive
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/IDS project - models and encoders/')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
#imports
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.preprocessing import LabelEncoder
import random
import ipywidgets as widgets
from IPython.display import display, clear_output
import pickle
import ipywidgets as widgets
from tensorflow.keras.models import load_model
print(tf.__version__)

2.17.1


In [10]:
# Loading in previously trained and saved models and encoders
model_species = load_model('model_species.keras')
with open("label_encoder_species.pkl", 'rb') as file:
    label_encoder_species = pickle.load(file)

# Dictionary to hold disease-specific models and encoders
disease_models_encoders = {}

# List of species and corresponding file names
species_files = {
    "Tomato": ("model_Tomato.keras", "label_encoder_Tomato.pkl"),
    "Potato": ("model_Potato.keras", "label_encoder_Potato.pkl"),
    "Apple": ("model_Apple.keras", "label_encoder_Apple.pkl"),
    "Grape": ("model_Grape.keras", "label_encoder_Grape.pkl"),
    "Peach": ("model_Peach.keras", "label_encoder_Peach.pkl"),
    "Cherry_(including_sour)": ("model_Cherry_(including_sour).keras", "label_encoder_Cherry_(including_sour).pkl"),
    "Corn_(maize)": ("model_Corn_(maize).keras", "label_encoder_Corn_(maize).pkl"),
    "Pepper,_bell": ("model_Pepper,_bell.keras", "label_encoder_Pepper,_bell.pkl"),
    "Strawberry": ("model_Strawberry.keras", "label_encoder_Strawberry.pkl")
}

# Load disease-specific models and encoders
for species, (model_file, encoder_file) in species_files.items():
    with open(encoder_file, 'rb') as file:
        label_encoder = pickle.load(file)
    disease_models_encoders[species] = [load_model(model_file), label_encoder]

# Load the 'all' model and encoder
model_all = load_model("model_all.keras")
with open("label_encoder_all.pkl", 'rb') as file:
    label_encoder_all = pickle.load(file)

# Check the loaded encoders
print(disease_models_encoders)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'Tomato': [<Sequential name=sequential_9, built=True>, LabelEncoder()], 'Potato': [<Sequential name=sequential_7, built=True>, LabelEncoder()], 'Apple': [<Sequential name=sequential_1, built=True>, LabelEncoder()], 'Grape': [<Sequential name=sequential_4, built=True>, LabelEncoder()], 'Peach': [<Sequential name=sequential_5, built=True>, LabelEncoder()], 'Cherry_(including_sour)': [<Sequential name=sequential_2, built=True>, LabelEncoder()], 'Corn_(maize)': [<Sequential name=sequential_3, built=True>, LabelEncoder()], 'Pepper,_bell': [<Sequential name=sequential_6, built=True>, LabelEncoder()], 'Strawberry': [<Sequential name=sequential_8, built=True>, LabelEncoder()]}


In [42]:
#Saving an image from camera
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image

def take_photo(filename='/tmp/photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [46]:
#Making the user interfrace
class_labels = ["Tomato", "Potato", "Apple", "Grape", "Peach", "Strawberry", "Corn", "Cherry", "Pepper"]

# Preprocessing function
def preprocess_image(img):
    img = cv2.resize(img, (256, 256))  # Resize image
    img = img / 255.0                  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Prediction function
def predict_image(img, selected_species):
    if selected_species == "Unknown": #If no species is specified, use the model_all (predicting species and disease together)
      model = model_all
      encoder = label_encoder_all
    else: #If species is specified, use the corresponding model
      #Three species have a cleaned name in the selection menu and a different name in the disease_models_encoders dictionary, so change the name if necessary
      if selected_species == "Cherry":
        selected_species = "Cherry_(including_sour)"
      elif selected_species == "Corn":
        selected_species = "Corn_(maize)"
      elif selected_species == "Pepper":
        selected_species = "Pepper,_bell"
      model = disease_models_encoders[selected_species][0]
      encoder = disease_models_encoders[selected_species][1]


    preprocessed_img = preprocess_image(img)
    predictions = model.predict(preprocessed_img)
    predicted_class = np.argmax(predictions)
    probability = predictions[0][predicted_class]
    predicted_class_name = encoder.inverse_transform([predicted_class])[0]
    print(f"Predicted class: {predicted_class_name}, probability: {probability:.2f}")


# Image upload callback
def on_upload_change(change):
    with output_area:
        clear_output()
        try:
            selected_species = species_dropdown.value
            uploaded_file = list(upload_button.value.values())[0]
            image_bytes = uploaded_file['content']
            img = cv2.imdecode(np.frombuffer(image_bytes, np.uint8), cv2.IMREAD_COLOR)
            result = np.argmax(predict_image(img, selected_species))
            cv2_imshow(cv2.resize(img, (300, 300)))
        except Exception as e:
            print(f"Error processing upload: {e}")

#Picture taken from camera
def on_camera_click(b):
    with output_area:
        clear_output()
        selected_species = species_dropdown.value
        photo = take_photo()
        img = cv2.imread(photo)
        cv2_imshow(cv2.resize(img, (300, 300)))

        result = predict_image(img, selected_species)


species_dropdown = widgets.Dropdown(
    options=['Unknown'] + class_labels,
    description='Species:'
)

upload_button = widgets.FileUpload(accept='image/*', multiple=False)
camera_button = widgets.Button(description='Take Picture')
output_area = widgets.Output()

upload_button.observe(on_upload_change, names='_counter')
camera_button.on_click(on_camera_click)

display(species_dropdown, upload_button, camera_button, output_area)

Dropdown(description='Species:', options=('Unknown', 'Tomato', 'Potato', 'Apple', 'Grape', 'Peach', 'Strawberr…

FileUpload(value={}, accept='image/*', description='Upload')

Button(description='Take Picture', style=ButtonStyle())

Output()